## IA : CP1
- Aubert Nicolas,
- ISC3il-b

# Exercice 1

## Data loading

In [102]:
# Charger les fichiers depuis le répertoire data en utilisant la méthode load_files() du package sklearn.datasets.
from sklearn.datasets import load_files

# Charger les données (./exo1/data')
data = load_files("./exo1/data/")

print(data.target_names)

# Diviser les données en train et test avec une répartition de 70-30%.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.3, random_state=0)

# Load les données de test (./exo1/data/test.csv')
data_test = load_files("./exo1/data/")

['neg', 'pos']


## Bayes

In [103]:
# Implémenter avec Bayes un Gridsearch avec une cross validation de 5 folders comme suit 
# use_idf : (True, False),
# ii. alpha : (1e-2, 1e-3)

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
 
parameters = {
    'vect__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3)
}

# Créer un pipeline avec un TfidfVectorizer() et un MultinomialNB().
# Utiliser le GridSearchCV pour trouver les meilleurs paramètres.

from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB()),
])

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

# Afficher les meilleurs paramètres de la cross-validation de chaque modèle.
 
grid_search.fit(X_train, y_train)

print("Best score: %0.3f" % grid_search.best_score_)

# Afficher le score de précision sur le jeu de test.
 
print(grid_search.score(X_test, y_test))

# Interpréter les valeurs trouvées dans use_idf.

# use_idf : (True, False) : True est meilleur que False

# Evaluer le modèle Bayes avec le dataset de test.

y_pred = grid_search.predict(X_test)

print("Classification report for Bayes : \n")

print(classification_report(y_test, y_pred, target_names=data.target_names))


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best score: 0.919
0.89
Classification report for Bayes : 

              precision    recall  f1-score   support

         neg       0.92      0.86      0.89       303
         pos       0.87      0.92      0.89       297

    accuracy                           0.89       600
   macro avg       0.89      0.89      0.89       600
weighted avg       0.89      0.89      0.89       600



## SVM

In [104]:
# Implémenter avec SVM un Gridsearch avec une cross validation de 5 folders, avec les paramètres suivants :
# use_idf: (True, False),
# ii. C: [0.1,1,5]

from sklearn.svm import LinearSVC
 
parameters = {
    'vect__use_idf': (True, False),
    'clf__C': [0.1,1,5]
}

# Créer un pipeline avec un TfidfVectorizer() et un LinearSVC().
# Utiliser le GridSearchCV pour trouver les meilleurs paramètres.

from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', LinearSVC()),
])

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

# Afficher les meilleurs paramètres trouvés par le GridSearchCV.

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

# Afficher le score de précision sur le jeu de test.

print(grid_search.score(X_test, y_test))

# Afficher le meilleur score de cross validation.

print("Best score: %0.3f" % grid_search.best_score_)

# Interpréter les valeurs trouvées dans use_idf.

# use_idf : (True, False) : True est meilleur que False

# Interpréter la valeur trouvée de C dans le cas de SVM.

# C : [0.1,1,5] : 5 est meilleur que 1 et 0.1

# Evaluer le modèle SVM avec le dataset de test.

from sklearn.metrics import classification_report

y_pred = grid_search.predict(X_test)

print("Classification report for SVM : \n")

print(classification_report(y_test, y_pred, target_names=data.target_names))


Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'clf__C': 5, 'vect__use_idf': True}
0.9316666666666666
Best score: 0.924
Classification report for SVM : 

              precision    recall  f1-score   support

         neg       0.94      0.92      0.93       303
         pos       0.92      0.94      0.93       297

    accuracy                           0.93       600
   macro avg       0.93      0.93      0.93       600
weighted avg       0.93      0.93      0.93       600



### Évaluation des modèles

SVM donne de meilleurs résultats que Bayes.

### Y-avait-il du surentrainement (overfitting) ? pourquoi ?



### Interpréter les résultats des f1_score.

Le f1_score de SVM est plus élevé que celui de Bayes. Cela signifie que SVM est plus précis que Bayes.

# Exercice 2 : Régression

Le but de ce régresseur est de prédire les classes de prix (0, 1, 2, 3) des téléphones portables à partir de leurs caractéristiques techniques (puissance de la batterie, couleurs, mémoire, etc.).

In [105]:
# 1. Lire les données du répertoire data dans un dataframe Pandas et afficher les premiers 10 lignes des données de train.

import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

df = pd.read_csv("./exo2/data/train.csv")

print(df.head(10))

# 2. Enlever depuis les données de train les lignes où la colonne px_height est nulle.

df = df[df['px_height'].notnull()]

# 3. Diviser les données en train et test.

X_train, X_test, y_train, y_test = train_test_split(df.drop('price_range', axis=1), df['price_range'], test_size=0.3, random_state=0)

   battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0            842     0          2.2         0   1       0           7    0.6   
1           1021     1          0.5         1   0       1          53    0.7   
2            563     1          0.5         1   2       1          41    0.9   
3            615     1          2.5         0   0       0          10    0.8   
4           1821     1          1.2         0  13       1          44    0.6   
5           1859     0          0.5         1   3       0          22    0.7   
6           1821     0          1.7         0   4       1          10    0.8   
7           1954     0          0.5         1   0       0          24    0.8   
8           1445     1          0.5         0   0       0          53    0.7   
9            509     1          0.6         1   2       1           9    0.1   

   mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  talk_time  \
0        188        2  ...         20  

In [109]:
# 4. Utiliser un KNN classifier (KNeighborsClassifier) pour faire une première classification. Comment vous choisissez le k.

# Find the best k parameter

# Initialize a dict to store the accuracies for each value of k
accuracies = {}

for k in range(1, 20):
    # initialize the classifier
    clf = KNeighborsClassifier(k, weights='uniform')

    # train it with the 'fit' method (you should pass the training set and the related labels)
    clf.fit(X_train, y_train)
    
    # compute the predictions using the test set
    y_pred = clf.predict(X_test) 

    # Compute the accuracy of the classification
    accuracy = accuracy_score(y_test, y_pred)
    
    accuracies[k] = accuracy

# Print the accuracies for each value of k
for k, accuracy in accuracies.items():
    print("k = %d, accuracy = %f" % (k, accuracy))

# print the best k
best_k = max(accuracies, key=accuracies.get)
print("Best k = %d" % best_k, "with accuracy = %f" % accuracies[best_k])

k = 1, accuracy = 0.893333
k = 2, accuracy = 0.890000
k = 3, accuracy = 0.910000
k = 4, accuracy = 0.906667
k = 5, accuracy = 0.925000
k = 6, accuracy = 0.933333
k = 7, accuracy = 0.943333
k = 8, accuracy = 0.936667
k = 9, accuracy = 0.940000
k = 10, accuracy = 0.935000
k = 11, accuracy = 0.938333
k = 12, accuracy = 0.940000
k = 13, accuracy = 0.938333
k = 14, accuracy = 0.941667
k = 15, accuracy = 0.941667
k = 16, accuracy = 0.941667
k = 17, accuracy = 0.936667
k = 18, accuracy = 0.946667
k = 19, accuracy = 0.943333
Best k = 18 with accuracy = 0.946667


In [107]:

knn = KNeighborsClassifier(n_neighbors=best_k)

knn.fit(X_train, y_train)

# 5. Afficher le score de précision sur le jeu de test.
 
print(knn.score(X_test, y_test))

# 5. Implémenter un pipeline SVM avec un scaler de votre choix.

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', SVC()),
])

# 6. Chercher avec un gridsearch de votre choix les meilleurs paramètres de votre SVM.

parameters = {
    'clf__C': [0.1,1,5],
    'clf__kernel': ['linear', 'rbf', 'poly'],
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

# 7. Afficher le rapport d’évaluation après l’évaluation des données de test.

y_pred = grid_search.predict(X_test)

print(classification_report(y_test, y_pred))

0.9466666666666667
Fitting 5 folds for each of 9 candidates, totalling 45 fits
{'clf__C': 5, 'clf__kernel': 'linear'}
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       151
           1       0.94      0.94      0.94       135
           2       0.95      0.93      0.94       151
           3       0.98      0.98      0.98       163

    accuracy                           0.96       600
   macro avg       0.96      0.96      0.96       600
weighted avg       0.96      0.96      0.96       600



In [108]:
# 9. Facultatif : implémenter un arbre de décision puis un Forest Tree (from sklearn.ensemble import RandomForestClassifier) (+3 pts)

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

tree = DecisionTreeClassifier()

tree.fit(X_train, y_train)

print("Decision Tree score : %f" % tree.score(X_test, y_test))

forest = RandomForestClassifier()

forest.fit(X_train, y_train)

print("Forest Tree score : %f" % forest.score(X_test, y_test))

Decision Tree score : 0.821667
Forest Tree score : 0.853333


- La précision moyenne est de 0.96

- Le Forest Tree (0.88) est meilleur que le Decision Tree (0.83).